In [ ]:
# author: Tom Stone <tomstone@stanford.edu>
# author: Proloy Das <email:proloyd94@gmail.com>
# License: BSD (3-clause)
%matplotlib widget
%matplotlib widget
import os
import numpy as np
from scipy import signal, fft
from pathlib import Path

from matplotlib import pyplot
from lab1_utils import *

# rcParams are set to make the plots pretty (i.e., publication ready).
pyplot.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica",
    "figure.constrained_layout.use": True,
    "savefig.dpi": 300      # very imp for publications
})

We will create a random number generator, and seed it (the argument) to make the random number generation reproducible. 
That is, the numbers generarted will be 'random' in the probabilty measure sense, but they will be the same everytime you run the code. 

In [ ]:
rng = np.random.default_rng(12345)

## Generating a white noise signal
Create n=1024 time-samples of a Gaussian white noise process.
$$𝔼[𝑤_𝑘]=𝜇$$
$$Var(𝑤_𝑘 )=𝜎^2$$
$$Cov(𝑤_𝑘, 𝑤_𝑙 )=0  \text{ for } 𝑘≠𝑙$$
Assume $\mu = 0$. Hint: use `rng.normal()`

Also create the time indices corresponding the white noise sequence.

In [ ]:
wn = rng.normal(size=1024)
time_indices = np.arange(1024)

Plot the white noise againt time using matplotlib.

In [ ]:
fig1, ax = pyplot.subplots(figsize=(8, 2))
ax.plot(time_indices, wn, linewidth=0.5)
ax.set_xlabel('$t$')
ax.set_ylabel('$w_t$')
ax.set_ylim([-4, 4])
ax.set_title('White noise process')

Now we compute the autocovaraince sequence from the realization that we generated.
$$\hat{s}_k = \frac{1}{N}\sum_{i=1}^{N-\vert k\vert}(x_i-\hat{\mu})(x_{i+\vert k \vert} -\hat{\mu})$$
For that write a funtion with `compute_autocovaraince(signal, max_lag)` that takes in the signal and the maximum lag upto which it needs to compute the autocovaraince sequence as inputs, and returns the autocovaraince, and the associated time indices as output.

Note: This problem can be solved very easily with only a few line of code using `signal.correlate` funtion as instructed bewlow. 

In [ ]:
def compute_autocovaraince(x, max_lag=512):
    """Computes autocovarinane upto a given lag.
    
    Parameters:
        x:
            the signal
        max_lag: 
            maximum lag to consider for autocovaraince sequenc
    Returns:
        acov:
            the sample autocovariance, normalized by the number of samples.
        time_indices:
         integer shifts, i.e the x-axis for the autocovariance plot.
    """
    assert max_lag > 0, f"max_lag needs to be >0, received {max_lag}"
    n = wn.shape[-1] # length of time series
    assert max_lag < n, f"max_lag needs to be < signal length ({n}) , received {max_lag}"
    # Compute the mean, `mu`
    mu = x.sum(axis=-1) / n

    # Remove the mean from the sequence
    x = x - mu

    # Use `signal.correlate` function to compute the acov sequence
    # See https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.correlate.html
    # Use mode='full' which returns the full result, i.e. lag -n+1 to lag n-1
    acov = signal.correlate(x, x, mode='full', method='fft') / n
    # Create an index array corrsponding to the correlation values using `numpy.arange``
    indices = np.arange(-n+1, n)

    # Create a boolean selection to extract indices within range[-max_lag, max_lag].
    # Use `numpy.logical_and()`
    selection = np.logical_and(indices < max_lag+1, indices > -max_lag-1)
    return acov[selection], indices[selection]

We will now use the function to compute sample autocovariance sequnce, and compare with the true autocovaraince sequence using a plot.
$$s_k =\begin{cases}
        1 \text{, } k=0 \\
        0 \text{, otherwise}
        \end{cases}$$

In [ ]:
# Sampe autocorrelation
max_lag = wn.shape[-1] - 1
sample_acov, lags = compute_autocovaraince(wn, max_lag)

# True autocovaraince
true_acov = np.zeros_like(sample_acov)
true_acov[max_lag] = 1

fig1, ax = pyplot.subplots(figsize=(8, 2))
ax.plot(lags, sample_acov, linewidth=1, color='r', label='Sample')
ax.plot(lags, true_acov, linewidth=1, color='b', label='True')
ax.set_ylim([-0.2, 1.2])
ax.legend()
_ = ax.set_title('Autocorrelation sequence plot')

Now we are ready to compute the periodogram. Create a function with signature `compute_periodogram(x)` that computes the autocovaraince sequence, and takes its Fourier transform.
$$\hat{S}(f) = T \sum_{-N+1}^{N-1} \hat{s}_k\exp(-i2\pi k fT)$$
The function shall return the power spectral density (PSD) estimate, and associated (normalized) frequency points.

Note: Use `scipy.fft.fft()` function perform the transform.

In [ ]:
S_xx_est = np.abs(fft.fft(sample_acov))
S_xx_true = np.abs(fft.fft(true_acov))
freqs = np.linspace(0, 1, num=len(lags))

# Periodogram plot
fig1, ax = pyplot.subplots(figsize=(5, 2))
ax.plot(freqs, 10*np.log10(S_xx_est), linewidth=1, color='r', label='Sample')
ax.plot(freqs, 10*np.log10(S_xx_true), linewidth=1, color='b', label='True')
ax.set_ylim([-30, 20])
ax.set_xlim([0., 0.5])
ax.legend()
_ = ax.set_title('Periodogram plot')

Now, create a function with signature `compute_periodogram(x)` that uses the following formulae:
$$\hat{S}(f) = \frac{T}{N} \left\vert\sum_{k=0}^{N-1} x_k\exp(-i2\pi k fT)\right\vert^2$$
The function shall return the power spectral density (PSD) estimate, and associated (normalized) frequency points.

Note: Use `scipy.fft.fft()` function perform the transform.

In [ ]:
def compute_periodogram(x):
    """Compute periodogram using fft directly on the signal.
    Parameters:
        x:
            the signal
        max_lag: 
            maximum lag to consider for autocovaraince sequenc
    Returns:
        S_xx:
            periodogram.
        freqs:
         associated frequency (normalized) points.
    """
    n = x.shape[-1]
    # Compute the mean
    mu = x.sum() / n
    # remove the mean from the signal
    x = x - mu
    
    # Now use the `fft.fft()` funtion to perform the computation.
    S_xx = np.abs(fft.fft(x)) ** 2 / n
    freqs = np.linspace(0., 1, num=n)
    return S_xx, freqs

Compare the output of the `compute_periodogram()` to the previously computed periodograms. 

In [ ]:
S_xx, freqs = compute_periodogram(wn)
ax.plot(freqs, 10*np.log10(S_xx), linewidth=1, color='g', label='True')

# Observations?

1. 
2.
...

We will use the `compute periodogram` function extensively for the rest of the lab.


Before moving further in this notebook, we will take a look at the random processes that are more structured than white noise. 
Lets jump to the AR(2) notebook, and run the same analysis. But, for running same analysis will require us to copy the funtions we just wrote to AR(2) notebook. We will approach this copying a pythonic way. Instead of copying the functions to other notebooks, we will copy them to another python scipt named lab1_utils.py.

You might ask *Why?*. The reason behind this is that once the functions are there, we simply need to import the functions as required, e.g.,
```from lab_utils import compute_autocovariance```
instead of copying the code anymore. 

Less code duplication is your friend for combatting against grave mistakes in data analysis. It makes your analysis managable from one place, thus making mudane code verification work very quick.